In [150]:
import numpy as np
import geopandas as gpd
import pandas as pd

import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input

import Levenshtein

gpd.options.io_engine = "pyogrio" # faster io than with fiona

## Datasets
Importing each dataset as a seperate dataframe and cleaning said dataframe.

In [151]:
# Utilities

def map_cbs_to_na(df):
    df = df.map(lambda x: np.nan if x == '-99999999' else x)
    df = df.map(lambda x: np.nan if x == -99999999 else x)
    df = df.map(lambda x: np.nan if x == '' else x)
    df = df.map(lambda x: np.nan if x == '.' else x)
    return df

def count_total_na(cols, df):
    return sum([df[col].isna().sum() for col in cols])

def select(r, df, col, code):
    if not np.isnan(r[col]): return r[col]
    i, sel = 0, df[df[code] == r[code]][col]
    while i < len(sel): 
        if not np.isnan(sel.iloc[i]):
            return sel.iloc[i]
        i += 1
    return np.NaN

def infer_cbs(infer_df, infer_cols, wk_file, gm_file):
    infer_df = map_cbs_to_na(infer_df)
    print(f'missing before inference: {count_total_na(infer_cols, infer_df)} \
          (ie {infer_df[infer_cols[0]].isna().sum()} for col 0)')
    
    for file_name, code in [(wk_file, 'WK_CODE'), (gm_file, 'GM_CODE')]:
        df = gpd.read_file(file_name)
        df = map_cbs_to_na(df)
        
        for col in infer_cols:
            infer_df[col] = [select(r, df, col, code) for _, r in infer_df.iterrows()]
        print(f"after inferring via {code}: {count_total_na(infer_cols, infer_df)}")
        
    return infer_df


def infer_bu_code(df, cols, stat_filter):
    to_district = lambda bu_code: f"WK{bu_code[2:-2]}  "
    to_municipality = lambda bu_code: f"GM{bu_code[2:-4]}    "
    inference_tracker = {'na': 0}
    col_tracker = {}

    new_lists = {col: [] for col in cols}
    def retrieve_values(row):
        for k in new_lists.keys():
            v = row[k]
            new_lists[k].append(row[k])

    def infer_value(row, col):
        for cbs_code in [to_district(row['BU_CODE']), to_municipality(row['BU_CODE'])]:
            new_v = df[df['BU_CODE'] == cbs_code][col]#.iloc[0] 
            new_v = new_v.iloc[0] if len(new_v) > 0 else np.NaN
            if not np.isnan(new_v): 
                return new_v, cbs_code[:2]
        return new_v, 'na'

    for _, r in df.iterrows():
        retrieve_values(r)
        if r['BU_CODE'][:2] != 'BU': continue

        for k in new_lists.keys():
            if np.isnan(new_lists[k][-1]): 
                new_lists[k][-1], inf_method = infer_value(r, k)
                if not stat_filter:
                    inference_tracker[inf_method] = inference_tracker.get(inf_method, 0) + 1
                    col_tracker[k] = col_tracker.get(k, 0) + 1
                elif r['BU_CODE'] in stat_filter:
                    inference_tracker[inf_method] = inference_tracker.get(inf_method, 0) + 1
                    col_tracker[k] = col_tracker.get(k, 0) + 1
                    
    for k, v in new_lists.items():
        df[k] = v
    print(f"Inferred a total of {sum(inference_tracker.values())-inference_tracker['na']}\
           values over {len(cols)} columns; {inference_tracker['na']} values still na. {'filtered' if stat_filter else ''}\n\
          (inference_tracker: {inference_tracker})\n\
          (col_tracker: {col_tracker})")
    return df

In [197]:
# * reading and cleaning neighbourhood core statistics 2022 (kern '22)

# reading in and filtering to zuid-holland municipalities
k22_gdf = gpd.read_file('data/zh/base/CBS_BUURTKAART_2022_VERSIE2.shp')
zh_municipalities_22 = ["Alblasserdam", "Albrandswaard", "Alphen aan den Rijn", "Barendrecht", "Bodegraven-Reeuwijk", "Capelle aan den IJssel", "Delft", "Dordrecht", "Goeree-Overflakkee", "Gorinchem", "Gouda", "'s-Gravenhage", "Hardinxveld-Giessendam", "Hellevoetsluis", "Hendrik-Ido-Ambacht", "Hillegom", "Hoeksche Waard", "Kaag en Braassem", "Katwijk", "Krimpen aan den IJssel", "Krimpenerwaard", "Lansingerland", "Leiden", "Leiderdorp", "Leidschendam-Voorburg", "Lisse", "Maassluis", "Midden-Delfland", "Molenlanden", "Nieuwkoop", "Nissewaard", "Noordwijk", "Oegstgeest", "Papendrecht", "Pijnacker-Nootdorp", "Ridderkerk", "Rijswijk", "Rotterdam", "Schiedam", "Sliedrecht", "Teylingen", "Vlaardingen", "Voorne aan Zee", "Voorschoten", "Waddinxveen", "Wassenaar", "Westland", "Zoetermeer", "Zoeterwoude", "Zuidplas", "Zwijndrecht"]
zh_municipalities_22 += ["Brielle", "Hellevoetsluis", "Westvoorne"]
k22_gdf = k22_gdf[k22_gdf['GM_NAAM'].isin(zh_municipalities_22)]
# removing water
k22_gdf = k22_gdf[k22_gdf['H2O'] == 'NEE']
# enforcing a few datatypes:
for col in ["AANT_INW", "AANT_MAN", "AANT_VROUW", "AUTO_TOT"]:
    k22_gdf[col] = k22_gdf[col].astype('int')

print(f"Baseline number of neighbourhoods: {k22_gdf.shape[0]}")

#designating useful columns from this gdf
k22_cols = ["AANT_INW", "AANT_MAN", "AANT_VROUW", "P_00_14_JR", "P_15_24_JR", "P_25_44_JR", "P_45_64_JR", "P_65_EO_JR", "P_KOOPWON", "P_HUURWON", "AUTO_TOT", "AF_ARTSPR", "AF_ZIEK_I", "AF_SUPERM", "AF_TREINST", "AF_OVERST", "AF_ONDBAS", "AF_ONDVRT", "AF_BIBLIO", "OPP_LAND"]
# reprojecting geometry
k22_gdf.to_crs('EPSG:28992', inplace=True)
#inferring missing values
k22_gdf = infer_cbs(k22_gdf, k22_cols, 
                    "data/wijkbuurtkaart_2022_v2/wijken_2022_v2.shp",
                    "data/wijkbuurtkaart_2022_v2/gemeenten_2022_v2.shp")
# normalising population size based features, updating k22_cols to match
k22_cols = ["AANT_INW", "OPP_LAND", "P_MALE_ISH", "P_FEM_ISH", "P_00_14_JR", "P_15_24_JR", "P_25_44_JR", "P_45_64_JR", "P_65_EO_JR", "P_KOOPWON", "P_HUURWON", "CARS_PP", "AF_ARTSPR", "AF_ZIEK_I", "AF_SUPERM", "AF_TREINST", "AF_OVERST", "AF_ONDBAS", "AF_ONDVRT", "AF_BIBLIO"]
k22_gdf = k22_gdf.assign(P_MALE_ISH = k22_gdf['AANT_MAN']/k22_gdf['AANT_INW'],
                         P_FEM_ISH = k22_gdf['AANT_VROUW']/k22_gdf['AANT_INW'],
                         CARS_PP = k22_gdf['AUTO_TOT']/k22_gdf['AANT_INW'])#cars per person
for new_column in ["P_MALE_ISH", "P_FEM_ISH", "CARS_PP"]:
    k22_gdf.loc[k22_gdf['AANT_INW'] == 0, new_column] = 0 # removing zero division errors

k22_gdf[['BU_CODE', 'WK_CODE', 'GM_CODE'] + k22_cols].head()

Baseline number of neighbourhoods: 2346
missing before inference: 3719           (ie 0 for col 0)
after inferring via WK_CODE: 352
after inferring via GM_CODE: 0


,BU_CODE,WK_CODE,GM_CODE,AANT_INW,OPP_LAND,P_MALE_ISH,P_FEM_ISH,P_00_14_JR,P_15_24_JR,P_25_44_JR,...,P_HUURWON,CARS_PP,AF_ARTSPR,AF_ZIEK_I,AF_SUPERM,AF_TREINST,AF_OVERST,AF_ONDBAS,AF_ONDVRT,AF_BIBLIO
103,BU04893053,WK048930,GM0489,10,71,0.500000,0.500000,15.0,12.0,22.0,...,9.0,6.500000,1.0,2.2,1.6,1.8,8.8,1.2,1.3,1.6
109,BU06380006,WK063800,GM0638,80,1,0.562500,0.437500,8.0,8.0,35.0,...,36.0,0.687500,0.1,7.0,0.1,3.6,6.2,0.3,3.5,0.4
110,BU05230301,WK052303,GM0523,1815,24,0.490358,0.509642,20.0,11.0,28.0,...,52.0,0.415978,4.0,7.4,0.3,1.3,19.0,0.6,5.5,0.5
113,BU06220504,WK062205,GM0622,4635,44,0.486516,0.514563,24.0,10.0,31.0,...,67.0,0.374326,0.6,2.2,0.4,4.6,9.3,0.6,0.5,1.9
114,BU15251103,WK152511,GM1525,1430,27,0.486014,0.513986,15.0,9.0,22.0,...,49.0,0.430070,0.8,1.8,0.6,0.9,8.4,0.4,0.4,1.5


In [178]:

# * reading and cleaning neighbourhood core statistics 2021 (kern '21)

# reading in and filtering to zuid-holland municipalities
k21_gdf = gpd.read_file('data/wijkbuurtkaart_2021_v3/buurten_2021_v3.shp')
k21_gdf = k21_gdf[k21_gdf['GM_NAAM'].isin(zh_municipalities_22)]
# removing water
k21_gdf = k21_gdf[k21_gdf['H2O'] == 'NEE']

print(f"Baseline number of neighbourhoods: {k21_gdf.shape[0]}")

# designating useful columns from this gdf
k21_cols = ['INK_ONTV2', 'A_OPL_LG', 'A_OPL_MD', 'A_OPL_HG'] #'INK_INW2', 
# reprojecting geometry
k21_gdf.to_crs('EPSG:28992', inplace=True)
# inferring missing values
k21_gdf = infer_cbs(k21_gdf, k21_cols, 
                    "data/wijkbuurtkaart_2021_v3/wijken_2021_v3.shp",
                    "data/wijkbuurtkaart_2021_v3/gemeenten_2021_v3.shp")
# normalising population size based variables, adjusting k21_cols to match
k21_cols = ['INK_ONTV2', 'P_OPL_LG', 'P_OPL_MD', 'P_OPL_HG'] #'INK_INW2', 
k21_gdf = k21_gdf.assign(P_OPL_LG = k21_gdf['A_OPL_LG']/k21_gdf['AANT_INW'],
                         P_OPL_MD = k21_gdf['A_OPL_MD']/k21_gdf['AANT_INW'],
                         P_OPL_HG = k21_gdf['A_OPL_HG']/k21_gdf['AANT_INW'])
k21_gdf[['BU_CODE', 'WK_CODE', 'GM_CODE'] + k21_cols].head()

Baseline number of neighbourhoods: 2315
missing before inference: 3252           (ie 2033 for col 0)
after inferring via WK_CODE: 676
after inferring via GM_CODE: 0


,BU_CODE,WK_CODE,GM_CODE,INK_ONTV2,P_OPL_LG,P_OPL_MD,P_OPL_HG
5731,BU04820101,WK048201,GM0482,36.0,0.179775,0.280899,0.213483
5732,BU04820102,WK048201,GM0482,36.0,0.255639,0.285714,0.105263
5733,BU04820103,WK048201,GM0482,36.0,0.175000,0.375000,0.168750
5734,BU04820104,WK048201,GM0482,36.0,0.207547,0.358491,0.169811
5735,BU04820105,WK048201,GM0482,36.0,0.153846,0.350427,0.170940


In [163]:

# * reading and cleaning green/grey (greenspace) dataset

# reading in file
gg_gdf = gpd.read_file('data/KEA_BASISKAART_GROEN_EN_GRIJS_PER_BUURT/KEA_BASISKAART_GROEN_EN_GRIJS_PER_BUURT.shp')
# adjusting column names to match k2x_gdf's
gg_gdf = gg_gdf.rename(columns={'Buurtcode': 'BU_CODE', 
                                'Buurtnaam': 'BU_NAAM',
                                'Gemeentena': 'GM_NAAM',
                                'perc_open0': 'P_PUB_TREES', 
                                'perc_open1': 'P_PUB_GREEN', 
                                'perc_open2': 'P_PUB_GREY'})
# defining useful columns for later use
gg_cols = ['P_PUB_TREES', 'P_PUB_GREEN', 'P_PUB_GREY']
#reprojecting geometry
gg_gdf.to_crs('EPSG:28992', inplace=True)

print(gg_gdf.shape)
print('missing values:', gg_gdf[gg_cols[0]].isna().sum())
gg_gdf.head(1)

(2313, 65)
missing values: 0


,BU_CODE,BU_NAAM,Gemeenteco,GM_NAAM,buurt_opp_,perc_bebou,perc_openb,perc_overi,perc_priva,perc_trans,...,Percentage,Percentag0,Percentag1,Oppervlakt,Percentag2,Stedelijk_,GroenBinne,GroenBinn0,Oppervlak0,geometry
0,BU19781405,Noordeloos-Overslingeland,GM1978,Molenlanden,168563.85,13.38,16.67,0.0,40.55,3.98,...,55.07,28.61,26.46,58753.14895934137),26.37,NaN,78,76,222045.158576,"POLYGON ((123447.543 432673.262, 123444.820 43..."


In [155]:
u_df = pd.read_csv('data/uitkeringen22v2.csv', sep=";")
u_cols = ['P_UIT_WH', 'P_UIT_BS', 'P_UIT_WAO']

u_df = u_df.rename(columns={"WijkenEnBuurten": "BU_CODE",
                            'Werkloosheidsuitkering_9':'P_UIT_WH', 
                            'Bijstandsuitkering_10':'P_UIT_BS', 
                            'Arbeidsongeschiktheidsuitkering_11':'P_UIT_WAO'})
u_df = u_df[u_df['Perioden'] == '2022MM12'].drop_duplicates('BU_CODE')
u_df = map_cbs_to_na(u_df)
u_df = infer_bu_code(u_df, u_cols, list(k22_gdf['BU_CODE']))

u_df.head(2)

Inferred a total of 957           values over 3 columns; 0 values still na. filtered
          (inference_tracker: {'na': 0, 'WK': 864, 'GM': 93})
          (col_tracker: {'P_UIT_WH': 319, 'P_UIT_BS': 319, 'P_UIT_WAO': 319})


,ID,BU_CODE,Perioden,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingWijkenEnBuurten_4,P_UIT_WH,P_UIT_BS,P_UIT_WAO,AOWUitkering_12
3,3,NL00,2022MM12,,,NL00,NaN,NaN,NaN,NaN,NaN
7,7,NL01,2022MM12,,,NL01,NaN,1.0,3.0,5.0,22.0


In [199]:

# * reading and cleaning labour participation dataset
# reading in dataset
l_df = pd.read_csv('data/arbeid22.csv', sep=';')
l_cols = ['P_WORKING']

l_df = l_df[l_df['Leeftijd'] == 52052] #filtering to age range 15-75
l_df = l_df.rename(columns={"WijkenEnBuurten": "BU_CODE",
                            'NettoArbeidsparticipatie_3':'P_WORKING'})
l_df = map_cbs_to_na(l_df)
l_df = infer_bu_code(l_df, l_cols, list(k22_gdf['BU_CODE']))

l_df.head(2)

Inferred a total of 578           values over 1 columns; 0 values still na. filtered
          (inference_tracker: {'na': 0, 'WK': 485, 'GM': 93})
          (col_tracker: {'P_WORKING': 578})


,ID,Geslacht,Leeftijd,BU_CODE,P_WORKING
0,0,T001038,52052,NL01,71.0
1,1,T001038,52052,GM1680,69.0


In [158]:

# * reading and cleaning crime dataset
# reading in dataset
c_df = pd.read_csv('data/crime_v2_2023.csv', sep=';')
# pivoting df such that values per crime for the entire year are summed up
c_df = pd.pivot_table(c_df, 
                      values='GeregistreerdeMisdrijven_1', 
                      index=['WijkenEnBuurten'], 
                      columns=['SoortMisdrijf'], aggfunc='sum')
# creating new column to represent physical crime in 2023 per neighbourhood
c_df = c_df.assign(PHYS_CRIME=c_df['0.0.0 ']-c_df['1.3.1 ']-c_df['3.7.4 ']) 
# cleaning up df shape
c_df = c_df.reset_index()[['WijkenEnBuurten', 'PHYS_CRIME']].rename(
    columns={'WijkenEnBuurten': 'BU_CODE'})
c_df.index.names = ['id']

print(f"Crime, number of neighbourhoods: {c_df.shape}")
c_df.head(3)

Crime, number of neighbourhoods: (18119, 2)


SoortMisdrijf,BU_CODE,PHYS_CRIME
id,,
0,BU00000000,3563
1,BU00140000,1398
2,BU00140001,2336


## Merging datasets

In [166]:

# * utility functions
def spatial_overlap_merge(gdf_l, gdf_r, unique_col='BU_CODE'):
    gdf = gpd.overlay(gdf_l, gdf_r, how='intersection', keep_geom_type=True)
    gdf['area'] = gdf.geometry.area
    gdf.sort_values(by='area', inplace=True)
    gdf.drop_duplicates(subset=unique_col, keep='last', inplace=True)
    gdf.drop(columns=['area'], inplace=True)
    return gdf

def reset_geometry(gdf, geom_gdf, id_col='BU_CODE'):
    gdf = gdf.merge(geom_gdf[[id_col, 'geometry']], how='left', on=id_col,
                    suffixes=['_l', '_r'])
    gdf = gdf.drop(columns=['geometry_l'])
    gdf = gdf.rename(columns={'geometry_r': 'geometry'})
    gdf = gdf.set_geometry('geometry')
    return gdf
    



def find_crime_val(c_df, bu_code):
    i, sel = 0, c_df[c_df['BU_CODE'] == bu_code]['PHYS_CRIME']
    while i < len(sel):
        return sel.iloc[i]
    return np.NaN
    

def infer_crimes(df, c_df, manual_annotation_csv, sum_override):
    manual_bu_conversions_df = pd.read_csv(manual_annotation_csv)
    candidate_bu_codes = list(manual_bu_conversions_df['BU_CODE'])
    
    new_values = []
    for _, r in df.iterrows():
        if not np.isnan(r['PHYS_CRIME']): 
            new_values.append(r['PHYS_CRIME'])
            continue
        if r['BU_CODE'] in sum_override.keys():
            new_values.append(sum([int(find_crime_val(c_df, bu) )
                                   for bu in sum_override[r['BU_CODE']]]))
            continue
        if r['BU_CODE'] in candidate_bu_codes:
            new_bu = manual_bu_conversions_df[manual_bu_conversions_df['BU_CODE'] == r['BU_CODE']]['NEW_BU'].iloc[0]
            new_values.append(find_crime_val(c_df, new_bu))
            continue
        new_values.append(np.NaN)
    df['PHYS_CRIME'] = new_values
    return df

In [200]:

# * the actual merge

# merging core statistic datasets
k22_gdf = k22_gdf[['BU_CODE', 'BU_NAAM', 'GM_CODE', 'GM_NAAM', 'geometry'] + k22_cols]
k21_gdf = k21_gdf[['geometry'] + k21_cols]
merged_gdf = spatial_overlap_merge(k22_gdf, k21_gdf)
# merging with greenspace dataset
gg_gdf = gg_gdf[['geometry'] + gg_cols]
merged_gdf = spatial_overlap_merge(merged_gdf, gg_gdf)
# merging with financial support dataset
u_df = u_df[['BU_CODE'] + u_cols]
merged_gdf = merged_gdf.merge(u_df, how='left', on='BU_CODE')
# merging with labour participation dataset
l_df = l_df[['BU_CODE'] + l_cols]
merged_gdf = merged_gdf.merge(l_df, how='left', on='BU_CODE')
# merging with crime dataset
merged_gdf = merged_gdf.merge(c_df, how='left', on='BU_CODE')
# inferring missing values
crime_manual_annotation_csv = 'data/missing_vals_crime.csv'
crime_joined_together_neighbourhoods = {
    #containing_neighbourhood: listlike(subsections)
    'BU06060906': ('BU06061006', 'BU06061002'),
    'BU06060907': ('BU06061003', 'BU06061004')}
merged_gdf = infer_crimes(merged_gdf, c_df, crime_manual_annotation_csv, crime_joined_together_neighbourhoods)
#cleaning up
merged_gdf = reset_geometry(merged_gdf, k22_gdf)


c:\Users\flori\miniconda3\envs\ssml2024\Lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
c:\Users\flori\miniconda3\envs\ssml2024\Lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union


In [201]:

# * saving merge
merged_gdf.to_file('out/complete_merge.shp')

C:\Users\flori\AppData\Local\Temp\ipykernel_13344\4075813199.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_gdf.to_file('out/complete_merge.shp')
c:\Users\flori\miniconda3\envs\ssml2024\Lib\site-packages\pyogrio\raw.py:530: RuntimeWarning: Normalized/laundered field name: 'P_PUB_TREES' to 'P_PUB_TREE'
  ogr_write(
c:\Users\flori\miniconda3\envs\ssml2024\Lib\site-packages\pyogrio\raw.py:530: RuntimeWarning: Normalized/laundered field name: 'P_PUB_GREEN' to 'P_PUB_GREE'
  ogr_write(


## Reviewing merge

In [202]:
for col in merged_gdf.columns:
    print(col, merged_gdf[col].isna().sum())

BU_CODE 0
BU_NAAM 0
GM_CODE 0
GM_NAAM 0
AANT_INW 0
OPP_LAND 0
P_MALE_ISH 0
P_FEM_ISH 0
P_00_14_JR 0
P_15_24_JR 0
P_25_44_JR 0
P_45_64_JR 0
P_65_EO_JR 0
P_KOOPWON 0
P_HUURWON 0
CARS_PP 0
AF_ARTSPR 0
AF_ZIEK_I 0
AF_SUPERM 0
AF_TREINST 0
AF_OVERST 0
AF_ONDBAS 0
AF_ONDVRT 0
AF_BIBLIO 0
INK_ONTV2 0
P_OPL_LG 0
P_OPL_MD 0
P_OPL_HG 0
P_PUB_TREES 0
P_PUB_GREEN 0
P_PUB_GREY 0
P_UIT_WH 0
P_UIT_BS 0
P_UIT_WAO 0
P_WORKING 0
PHYS_CRIME 6
geometry 0
